<a href="https://colab.research.google.com/github/chinmay404/FineTuned-DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit-CrewAi-Docs/blob/main/fine_tune_llm_using_lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git
!pip install wandb
!pip install -U bitsandbytes
!pip install -U transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Login With wandb and HuggingFace


In [5]:
from huggingface_hub import login
import wandb
from google.colab import userdata


wb_token = userdata.get('wandb')
wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tune-DeepSeek-R1-CREAW-AI-UNSLOTH',
    job_type="training",
    anonymous="allow"
)

huggingface_token = userdata.get('HUGGIN_FACE')
login(huggingface_token)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: chinmaypisal (chinmaypisal-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [ ]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import torch
import torch.nn as nn
#import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM , BitsAndBytesConfig



compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )




model = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    # load_in_8bit=True,
    quantization_config=bnb_config,
    device_map={"": 0},
)


tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
tokenizer.pad_token = tokenizer.eos_token

RuntimeError: Failed to import transformers.models.qwen2.modeling_qwen2 because of the following error (look up to see its traceback):
cannot import name 'is_decord_available' from 'transformers.utils' (/usr/local/lib/python3.11/dist-packages/transformers/utils/__init__.py)

In [ ]:
model.parameters()

<generator object Module.parameters at 0x7b52292843c0>

In [ ]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear4bit(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear4bit(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear4bit(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear4bit(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear4bit(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training


original_model = prepare_model_for_kbit_training(model)

In [ ]:
config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"
    ],
    bias="none",
    lora_dropout=0,
    task_type="CAUSAL_LM",
    random_state=3407,
    use_rslora=False,
    loftq_config=None
)
model.gradient_checkpointing_enable()

peft_model = get_peft_model(model, config)

In [ ]:
peft_model.print_trainable_parameters()

trainable params: 5,963,776 || all params: 1,783,051,776 || trainable%: 0.3345


# Loading Dataset

In [10]:
train_prompt = """
below is an instruction that describes a task , paired with an input that provides further context.
write a response that appropriately completes the response.
before answering think carefully . Create Chain of thoughts to ensure logical and accurate response.

### Instruction :
You Are A coder who know Creaw AI Framework and know how to code , Give Proper Explanation of code and solve a problem
please Answer The Followig Question

### Question :
{}

### Answer  :
{}


"""

In [11]:
EOS_token = tokenizer.eos_token



def format_prompt(question , answer):
    text = ""
    text += train_prompt.format(question , answer)
    text += EOS_token
    return text

In [13]:
import pandas as pd

csv_file = 'crew_ai_docs_synthetic_data.csv'
df = pd.read_csv(csv_file)
print(df.head())
formated_prompts = {"text": []}
for index, row in df.iterrows():
    question = row['Question']
    answer = row['Answer']
    formated_prompts["text"].append(format_prompt(question , answer))

                  File  Batch_Number  \
0  concepts_agents.txt             1   
1  concepts_agents.txt             1   
2  concepts_agents.txt             1   
3  concepts_agents.txt             1   
4  concepts_agents.txt             1   

                                            Question  \
0  How do I create an agent named AgentName using...   
1  How do I create an agent named AgentName using...   
2  How do I create an agent named AgentName using...   
3  How do I create an agent named AgentName using...   
4  What causes an error when trying to implement ...   

                                              Answer  Code  
0  {\n    "agents": [\n        {\n            "na...   NaN  
1  {\n  "crewai agents": {\n    "agent role": {\n...   NaN  
2  {\n    "agents": [\n        {\n            "@c...   NaN  
3  {\n    "agentDefinition": {\n        "role": "...   NaN  
4  {\n  "agents": {\n    "researcher": {\n      "...   NaN  


In [14]:
from datasets import Dataset

formated_prompts_dataset = Dataset.from_dict(formated_prompts)

train_test_split = formated_prompts_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=2048)

train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/427 [00:00<?, ? examples/s]

Map:   0%|          | 0/107 [00:00<?, ? examples/s]

In [ ]:
train_dataset.set_format("torch", columns=["input_ids", "attention_mask"])
eval_dataset.set_format("torch", columns=["input_ids", "attention_mask"])

# Traning

In [ ]:
from transformers import TrainingArguments
import time

output_dir = f'./peft-crew-ai-traning-{str(int(time.time()))}'
import transformers

peft_training_args = TrainingArguments(
    no_cuda=False,
    output_dir = output_dir,
    warmup_steps=1,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    max_steps=1000,
    learning_rate=2e-4,
    optim="adamw_torch_8bit",
    logging_steps=25,
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=25,
    evaluation_strategy="steps",
    eval_steps=25,
    do_eval=True,
    gradient_checkpointing=True,
    report_to="none",
    overwrite_output_dir = 'True',
    group_by_length=True,
    fp16=True,
)

peft_model.config.use_cache = False

peft_trainer = transformers.Trainer(
    model=peft_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=peft_training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

In [ ]:
peft_trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 9.27 GiB. GPU 0 has a total capacity of 14.74 GiB of which 3.30 GiB is free. Process 11480 has 11.43 GiB memory in use. Of the allocated memory 10.49 GiB is allocated by PyTorch, and 841.24 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

# USing A Lot Of GPU Trying Unsloth

In [8]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 3.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.6/187.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.3/318.3 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 834.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.2/253.2 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.

In [9]:
from unsloth import FastLanguageModel
import torch


max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.12: Fast Qwen2 patching. Transformers: 4.50.0.dev0.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.81G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/6.78k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

In [15]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None
)

Unsloth 2025.2.12 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [16]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported


trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,

    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        # num_train_epochs=1
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="wandb",

    ),
)

Applying chat template to train dataset (num_proc=2):   0%|          | 0/427 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/427 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/427 [00:00<?, ? examples/s]

Applying chat template to eval dataset (num_proc=2):   0%|          | 0/107 [00:00<?, ? examples/s]

Tokenizing eval dataset (num_proc=2):   0%|          | 0/107 [00:00<?, ? examples/s]

Tokenizing eval dataset (num_proc=2):   0%|          | 0/107 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [17]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 427 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 18,464,768
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
1,2.200400
2,2.194500
3,2.444600
4,1.964000
5,1.969300
6,1.922800
7,1.964900
8,2.105800
9,1.675400
10,1.931300


TrainOutput(global_step=60, training_loss=1.5388475636641183, metrics={'train_runtime': 433.2005, 'train_samples_per_second': 1.108, 'train_steps_per_second': 0.139, 'total_flos': 5388524252510208.0, 'train_loss': 1.5388475636641183})

# Saving Mode For Ollama and Other

In [18]:
model.save_pretrained_gguf("model", tokenizer)

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.
Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 1.8G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.02 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 28/28 [00:01<00:00, 23.64it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model.bin...
Done.


Unsloth: Converting qwen2 model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at model into q8_0 GGUF format.
The output location will be /content/model/unsloth.Q8_0.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model.bin'
INFO:hf-to-gguf:token_embd.weight,         torch.float16 --> Q8_0, shape = {1536, 151936}
INFO:hf-to-gguf:blk.0.attn_q.bias,         torch.float16 --> F32, shape = {1536}
INFO:h

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.


Unsloth: Conversion completed! Output location: /content/model/unsloth.Q8_0.gguf


# Testing The model

In [38]:
   from unsloth import FastLanguageModel
   from transformers import TextStreamer
   import torch

   max_seq_length = 2048
   dtype = None
   load_in_4bit = True

   print("Loading fine-tuned model")
   model, tokenizer = FastLanguageModel.from_pretrained(
       model_name="model",
      # max_seq_length=max_seq_length,
       dtype=dtype,
       load_in_4bit=load_in_4bit,
       token="token"
   )

   FastLanguageModel.for_inference(model)
   inputs = tokenizer(
       [
           """How do I create an agent named AgentName using the correct syntax in CrewAI ?
           """
       ],
       return_tensors="pt"
   ).to("cuda")

   text_streamer = TextStreamer(tokenizer)
   outputs = model.generate(**inputs, streamer=text_streamer, max_new_tokens=2046)
   print(tokenizer.batch_decode(outputs))


Loading fine-tuned model
==((====))==  Unsloth 2025.2.12: Fast Qwen2 patching. Transformers: 4.50.0.dev0.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
<｜begin▁of▁sentence｜>How do I create an agent named AgentName using the correct syntax in CrewAI ?
        <think>
Alright, so the user is asking about creating an agent named AgentName using CrewAI's correct syntax. Let me break this down.

First, I need to recall what CrewAI's agent syntax looks like. From what I remember, agents are created using the `create_agent` command. The basic syntax is `create_agent <name>`, where `<name>` is the name you want to give it. 

But wait, maybe there's more to it. Someti

In [ ]:
model.push_to_hub_gguf("chinmay1718/FineTuned-DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit-CrewAi-Docs", tokenizer, quantization_method = "q4_k_m")
model.push_to_hub_gguf("chinmay1718/FineTuned-DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit-CrewAi-Docs", tokenizer, quantization_method = "q8_0")

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.
Unsloth: Will remove a cached repo with size 1.6K


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.65 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 28/28 [00:01<00:00, 21.14it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving chinmay1718/FineTuned-DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit-CrewAi-Docs/pytorch_model.bin...


RuntimeError: *** Unsloth: Failed compiling llama.cpp using os.system(...) with error 2. Please report this ASAP!

In [ ]:
#model.save_pretrained_gguf("/content/drive/MyDrive/crew_ai_model", tokenizer, quantization_method = "q4_k_m")
#model.save_pretrained_gguf("/content/drive/MyDrive/crew_ai_model", tokenizer, quantization_method = "q8_0")
model.save_pretrained_gguf("/content/drive/MyDrive/f16", tokenizer, quantization_method = "f16")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.92 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 28/28 [00:00<00:00, 45.23it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving /content/drive/MyDrive/f16/pytorch_model.bin...
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['f16'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at /content/drive/MyDrive/f16 into f16 GGUF format.
The output location will be /content/drive/MyDrive/f16/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: f16
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model.bin'
INFO:hf-to-gguf:token_embd.weight,         torch.float16 --> F16, shape = {1536, 151936}
INFO:hf-to-gguf:blk.

In [ ]:
new_model="FineTuned-DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit-CrewAi-Docs"
trainer.model.save_pretrained(new_model)

config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
output_merged_dir = "/content/drive/MyDrive/Fine_Tune_LLM"

model.save_pretrained(output_merged_dir, safe_serialization = False)
tokenizer.save_pretrained(output_merged_dir)

('/content/drive/MyDrive/Fine_Tune_LLM/tokenizer_config.json',
 '/content/drive/MyDrive/Fine_Tune_LLM/special_tokens_map.json',
 '/content/drive/MyDrive/Fine_Tune_LLM/tokenizer.json')